In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import math
from datetime import timedelta

import plotly.express as px
import folium as flm
from plotly.subplots import make_subplots

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#colors
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801'

In [2]:
#offline plotly
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [3]:
#import all the prepared csvs
total_data = pd.read_csv('Data/Cleaned_Final_Global_Data.csv')
day_wise = pd.read_csv('Data/Day_Wise_Data.csv')
country_wise = pd.read_csv('Data/Country_Wise_Data.csv')
full_grouped = pd.read_csv('Data/Full_Grouped_Data.csv')

## Displaying current statistics of cases

In [4]:
temp = total_data.groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop=True)
tm=temp.melt(id_vars='Date', value_vars = ['Active','Deaths','Recovered'])
figure = px.treemap(tm,path=["variable"],values="value",height=300,width=1000,color_discrete_sequence=[act,dth,rec])
figure.data[0].textinfo = 'label+text+value'
figure.show()

## Let us display the graph for cases over time

In [5]:
temp = total_data.groupby('Date')['Recovered','Deaths','Active'].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],var_name='Case', value_name='Count')
# temp
figure = px.area(temp,x='Date',y='Count',color='Case')
figure.update_layout(xaxis_rangeslider_visible=True)
# figure.update_layout(yaxis_rangeslider_visible=True)
figure.show()

## Now let us analyse the current scenario of cases across the world

In [6]:
temp = total_data[total_data['Date']==max(total_data['Date'])]
# temp
global_map = flm.Map(location=[0,0],tiles='OpenStreetMap',min_zoom=2,max_zoom=10,zoom_start=0)
# global_map
for i in range (0,len(temp)):
    flm.Circle(
        location=[temp.iloc[i]['Lat'],temp.iloc[i]['Long']],
        color='red',
        fill='red',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country/Region'])+
                    '<li><bold>Province : '+str(temp.iloc[i]['Province/State'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['Confirmed'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Deaths']),
        radius = int(temp.iloc[i]['Confirmed'])
    ).add_to(global_map)
global_map

## TRY DOING A CHLOROPLETH MAP IF DEEPER INFO FOUND

## Cases over time in the world

In [41]:
temp = full_grouped
temp['Date'] = pd.to_datetime(temp['Date'])

figure = px.choropleth(temp,locations="Country/Region",locationmode='country names', color = np.log(full_grouped["Confirmed"]),
                       hover_name="Country/Region", animation_frame=full_grouped["Date"].dt.strftime('%Y-%m-%d'),
                       title="Cases globally over time", color_continuous_scale=px.colors.sequential.Plasma,projection='natural earth')
figure.update_layout(height=600,width=900)
figure.show()

In [8]:
# full_grouped.info()

## Cases over time

In [19]:
fig_c = px.bar(day_wise, x="Date", y="Confirmed", color_discrete_sequence = [act])
fig_d = px.bar(day_wise, x="Date", y="Deaths", color_discrete_sequence = [dth])
fig_r = px.bar(day_wise, x="Date", y="Recovered", color_discrete_sequence =[rec])
fig = make_subplots(rows=1, cols=3, shared_xaxes=False, horizontal_spacing=0.1,
                    subplot_titles=('Confirmed cases', 'Deaths reported','Recovered'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)
fig.add_trace(fig_r['data'][0], row=1, col=3)
fig.update_layout(height=480)
fig.update_layout(width=1000)
fig.show()

In [32]:
# day_wise.columns

In [25]:
fig_1 = px.line(day_wise, x="Date", y="Deaths / 100 Cases", color_discrete_sequence = [dth])
fig_2 = px.line(day_wise, x="Date", y="Deaths / 100 Recovered", color_discrete_sequence = ['#333333'])

fig = make_subplots(rows=1, cols=2, shared_xaxes=False, 
                    subplot_titles=('Deaths / 100 Cases','Deaths / 100 Recovered'))

fig.add_trace(fig_1['data'][0], row=1, col=1)
fig.add_trace(fig_2['data'][0], row=1, col=2)

fig.update_layout(height=480)
fig.show()

In [30]:
# day_wise.columns

In [28]:
fig_c = px.bar(day_wise, x="Date", y="New Confirmed", color_discrete_sequence = [act])
fig_d = px.bar(day_wise, x="Date", y="No. of countries", color_discrete_sequence = [dth])

fig = make_subplots(rows=1, cols=2, shared_xaxes=False, horizontal_spacing=0.1,
                    subplot_titles=('No. of new cases everyday', 'No. of countries'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.update_layout(height=480)
fig.show()

In [53]:
# country_wise.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New Confirmed,New Recovered,New Deaths,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Population,Cases / Million People,Confirmed today,Confirmed last week,1 week change,1 week % increase
0,Afghanistan,1279,42,179,1058,103,13,2,3.28,14.00,23.46,38742911.0,33.0,1279,840,439,52.26
1,Albania,663,27,385,251,29,29,0,4.07,58.07,7.01,2878420.0,230.0,663,518,145,27.99
2,Algeria,3007,407,1355,1245,97,151,5,13.54,45.06,30.04,43685618.0,69.0,3007,2268,739,32.58
3,Andorra,723,37,333,353,0,24,0,5.12,46.06,11.11,77240.0,9360.0,723,673,50,7.43
4,Angola,25,2,6,17,0,0,0,8.00,24.00,33.33,32644783.0,1.0,25,19,6,31.58


# Analysis of statistics of top 25 countries

In [56]:
fig_a = px.bar(country_wise.sort_values('Active').tail(25),x="Active",y="Country/Region",text='Active',orientation='h',color_discrete_sequence=[act])
fig_a.update_layout(title="Active Cases")
fig_a.show()

In [66]:
fig_c = px.bar(country_wise.sort_values('Confirmed').tail(25), x="Confirmed", y="Country/Region", 
               text='Confirmed', orientation='h', color_discrete_sequence = [cnf])
fig_d = px.bar(country_wise.sort_values('Deaths').tail(25), x="Deaths", y="Country/Region", 
               text='Deaths', orientation='h', color_discrete_sequence = [dth])
fig_r = px.bar(country_wise.sort_values('Recovered').tail(25), x="Recovered", y="Country/Region", 
               text='Recovered', orientation='h', color_discrete_sequence = [rec])
figure = make_subplots(rows=1,cols=3,shared_xaxes=False,horizontal_spacing=0.2,subplot_titles=('Cases Confirmed','Death Reports','Recovered'))
figure.add_trace(fig_c['data'][0],row=1,col=1)
figure.add_trace(fig_d['data'][0],row=1,col=2)
figure.add_trace(fig_r['data'][0],row=1,col=3)
figure.update_layout(height=600)
figure.show()

In [68]:
fig_dc = px.bar(country_wise.sort_values('Deaths / 100 Cases').tail(25), x="Deaths / 100 Cases", y="Country/Region", 
               text='Deaths / 100 Cases', orientation='h', color_discrete_sequence = ['#f38181'])
fig_rc = px.bar(country_wise.sort_values('Recovered / 100 Cases').tail(25), x="Recovered / 100 Cases", y="Country/Region", 
               text='Recovered / 100 Cases', orientation='h', color_discrete_sequence = ['#a3de83'])
figure = make_subplots(rows=1,cols=2,shared_xaxes=False,horizontal_spacing=0.2,subplot_titles=('Deaths per 100 Cases','Recovered per 100 Cases'))
figure.update_layout(height=600)
figure.add_trace(fig_dc['data'][0],row=1,col=1)
figure.add_trace(fig_rc['data'][0],row=1,col=2)
figure.show()

In [69]:
country_wise.columns

Index(['Country/Region', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'New Confirmed', 'New Recovered', 'New Deaths', 'Deaths / 100 Cases',
       'Recovered / 100 Cases', 'Deaths / 100 Recovered', 'Population',
       'Cases / Million People', 'Confirmed today', 'Confirmed last week',
       '1 week change', '1 week % increase'],
      dtype='object')

In [72]:
fig_nc = px.bar(country_wise.sort_values('New Confirmed').tail(25), x="New Confirmed", y="Country/Region", 
               text='New Confirmed', orientation='h', color_discrete_sequence = ['#c61951'])
temp = country_wise[country_wise['Population']>1000000]
fig_p = px.bar(temp.sort_values('Cases / Million People').tail(25), x="Cases / Million People", y="Country/Region", 
               text='Cases / Million People', orientation='h', color_discrete_sequence = ['#741938'])

figure = make_subplots(rows=1,cols=2,shared_xaxes=False, horizontal_spacing=0.2, subplot_titles=('New Cases Today','Cases per Million (Pop> 1 mil.)') )
figure.add_trace(fig_nc['data'][0],row=1,col=1)
figure.add_trace(fig_p['data'][0],row=1,col=2)
figure.update_layout(height=600)
figure.show()

In [73]:
fig_wc = px.bar(country_wise.sort_values('1 week change').tail(25), x="1 week change", y="Country/Region", 
               text='1 week change', orientation='h', color_discrete_sequence = ['#004a7c'])
temp = country_wise[country_wise['Confirmed']>100]
fig_pi = px.bar(temp.sort_values('1 week % increase').tail(25), x="1 week % increase", y="Country/Region", 
               text='1 week % increase', orientation='h', color_discrete_sequence = ['#005691'], 
                hover_data=['Confirmed last week', 'Confirmed'])
figure = make_subplots(rows=1,cols=2,shared_xaxes=False, horizontal_spacing=0.2, subplot_titles=('1 Week Change','1 Week % Increase') )
figure.add_trace(fig_wc['data'][0],row=1,col=1)
figure.add_trace(fig_pi['data'][0],row=1,col=2)
figure.update_layout(height=600)
figure.show()

In [81]:
fig = px.scatter(country_wise.sort_values('Deaths', ascending=False).iloc[:25, :], 
                 x='Confirmed', y='Deaths', color='Country/Region', size='Confirmed', height=700,
                 text='Country/Region', log_x=True, log_y=True, title='Deaths vs Confirmed (Scale is in log10)')
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)
fig.show()